In [1]:
# nuclio: ignore
import nuclio

In [2]:
%nuclio config kind = "job"
%nuclio config spec.image = "mlrun/ml-models"

%nuclio: setting kind to 'job'
%nuclio: setting spec.image to 'mlrun/ml-models'


In [3]:
import numpy as np
import pandas as pd
from mlrun.execution import MLClientCtx

def load_dataset(
        context: MLClientCtx,
        dataset: str,
        name: str = '',
        file_ext: str = 'parquet',
        params: dict = {}
) -> None:
    """Loads a scikit-learn toy dataset for classification or regression

    The following datasets are available ('name' : desription):

        'boston'          : boston house-prices dataset (regression)
        'iris'            : iris dataset (classification)
        'diabetes'        : diabetes dataset (regression)
        'digits'          : digits dataset (classification)
        'linnerud'        : linnerud dataset (multivariate regression)
        'wine'            : wine dataset (classification)
        'breast_cancer'   : breast cancer wisconsin dataset (classification)

    The scikit-learn functions return a data bunch including the following items:
    - data              the features matrix
    - target            the ground truth labels
    - DESCR             a description of the dataset
    - feature_names     header for data

    The features (and their names) are stored with the target labels in a DataFrame.

    For further details see https://scikit-learn.org/stable/datasets/index.html#toy-datasets

    :param context:    function execution context
    :param dataset:    name of the dataset to load
    :param name:       artifact name (defaults to dataset)
    :param file_ext:   output file_ext: parquet or csv
    :param params:     params of the sklearn load_data method
    """
    dataset = str(dataset)
    # reach into module and import the appropriate load_xxx function
    pkg_module = 'sklearn.datasets'
    fname = f'load_{dataset}'

    pkg_module = __import__(pkg_module, fromlist=[fname])
    load_data_fn = getattr(pkg_module, fname)

    data = load_data_fn(**params)
    feature_names = data['feature_names']

    # create the toy dataset
    xy = np.concatenate([data['data'], data['target'].reshape(-1, 1)], axis=1)
    if hasattr(feature_names, 'append'):
        # its a list
        feature_names.append('labels')
    else:
        # its an array
        feature_names = np.append(feature_names, 'labels')
    df = pd.DataFrame(data=xy, columns=feature_names)

    # log and upload the dataset
    context.log_dataset(name or dataset, df=df, format=file_ext, index=False)

In [4]:
# nuclio: end-code

### mlconfig

In [5]:
from mlrun import mlconf
import os
mlconf.dbpath = mlconf.dbpath or 'http://mlrun-api:8080'
mlconf.artifact_path = mlconf.artifact_path or f'{os.environ["HOME"]}/artifacts'

### save

In [6]:
from mlrun import code_to_function 
# create job function object from notebook code
fn = code_to_function("load_dataset")

# add metadata (for templates and reuse)
fn.spec.default_handler = "load_dataset"
fn.spec.description = "load a toy dataset from scikit-learn"
fn.metadata.categories = ["data-source", "ml"]
fn.metadata.labels = {"author": "yjb", "framework": "sklearn"}

fn.export("function.yaml")

[mlrun] 2020-05-03 21:00:05,529 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-05-03 21:00:05,606 function spec saved to path: function.yaml


## tests

In [7]:
if "V3IO_HOME" in list(os.environ):
    from mlrun import mount_v3io
    fn.apply(mount_v3io())
else:
    # is you set up mlrun using the instructions at https://github.com/mlrun/mlrun/blob/master/hack/local/README.md
    from mlrun.platforms import mount_pvc
    fn.apply(mount_pvc('nfsvol', 'nfsvol', '/home/joyan/data'))

In [8]:
from mlrun import NewTask    

task_params = {
    "name"   : "tasks load toy dataset", 
    "params" : {"dataset"     : "iris"}}

### run remotely

In [9]:
run = fn.run(NewTask(**task_params), artifact_path=mlconf.artifact_path)

[mlrun] 2020-05-03 21:00:05,684 starting run tasks load toy dataset uid=0619dd26b59842b89867fdb1cb390245  -> http://mlrun-api:8080
[mlrun] 2020-05-03 21:00:05,838 Job is running in the background, pod: tasks-load-toy-dataset-747mg
[mlrun] 2020-05-03 21:00:09,690 log artifact iris at /User/artifacts/iris.parquet, size: 5041, db: Y

[mlrun] 2020-05-03 21:00:09,724 run executed, status=completed
final state: succeeded


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...cb390245,0,May 03 21:00:09,completed,tasks load toy dataset,host=tasks-load-toy-dataset-747mgkind=jobowner=adminv3io_user=admin,,dataset=iris,,iris


to track results use .show() or .logs() or in CLI: 
!mlrun get run 0619dd26b59842b89867fdb1cb390245  , !mlrun logs 0619dd26b59842b89867fdb1cb390245 
[mlrun] 2020-05-03 21:00:12,058 run executed, status=completed


### or locally

In [10]:
from mlrun import run_local

In [11]:
for dataset in ["wine", "iris", "breast_cancer"]:
    run_local(handler=load_dataset,
              inputs={"dataset": dataset}, artifact_path=mlconf.artifact_path)

[mlrun] 2020-05-03 21:00:12,080 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-05-03 21:00:12,081 starting run mlrun-e8b13b-load_dataset uid=7a83c513f6fe4938b1dda31be0ed2260  -> http://mlrun-api:8080
[mlrun] 2020-05-03 21:00:12,106 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-05-03 21:00:12,612 log artifact wine at /User/artifacts/wine.parquet, size: 18006, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...e0ed2260,0,May 03 21:00:12,completed,mlrun-e8b13b-load_dataset,v3io_user=adminkind=handlerowner=adminhost=jupyter-6fd84575d4-m5dcx,dataset,,,wine


to track results use .show() or .logs() or in CLI: 
!mlrun get run 7a83c513f6fe4938b1dda31be0ed2260 --project default , !mlrun logs 7a83c513f6fe4938b1dda31be0ed2260 --project default
[mlrun] 2020-05-03 21:00:12,673 run executed, status=completed
[mlrun] 2020-05-03 21:00:12,680 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-05-03 21:00:12,681 starting run mlrun-c306a8-load_dataset uid=1f2918e9ac25470595ffebdf8a416f2b  -> http://mlrun-api:8080
[mlrun] 2020-05-03 21:00:12,697 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-05-03 21:00:12,751 log artifact iris at /User/artifacts/iris.parquet, size: 5049, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...8a416f2b,0,May 03 21:00:12,completed,mlrun-c306a8-load_dataset,v3io_user=adminkind=handlerowner=adminhost=jupyter-6fd84575d4-m5dcx,dataset,,,iris


to track results use .show() or .logs() or in CLI: 
!mlrun get run 1f2918e9ac25470595ffebdf8a416f2b --project default , !mlrun logs 1f2918e9ac25470595ffebdf8a416f2b --project default
[mlrun] 2020-05-03 21:00:12,792 run executed, status=completed
[mlrun] 2020-05-03 21:00:12,800 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-05-03 21:00:12,801 starting run mlrun-1eba12-load_dataset uid=d8baaa65a48448abbb8908dae8bc544f  -> http://mlrun-api:8080
[mlrun] 2020-05-03 21:00:12,823 warning!, server (0.4.6) and client (0.4.7) ver dont match
[mlrun] 2020-05-03 21:00:13,026 log artifact breast_cancer at /User/artifacts/breast_cancer.parquet, size: 146754, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...e8bc544f,0,May 03 21:00:12,completed,mlrun-1eba12-load_dataset,v3io_user=adminkind=handlerowner=adminhost=jupyter-6fd84575d4-m5dcx,dataset,,,breast_cancer


to track results use .show() or .logs() or in CLI: 
!mlrun get run d8baaa65a48448abbb8908dae8bc544f --project default , !mlrun logs d8baaa65a48448abbb8908dae8bc544f --project default
[mlrun] 2020-05-03 21:00:13,102 run executed, status=completed
